In [9]:
import pandas as pd
import sqlite3
import csv
# Define the path to the SQLite database
db_path = r'C:\Users\hofong\Desktop\python\1a Python working station\Database\TDC_Fair_database.db'
BRN_path = r'C:\Users\hofong\Desktop\python\1a Python working station\participation check\brn.csv'



BRN_path = r'C:\Users\hofong\Desktop\python\1a Python working station\participation check\brn.csv'
df = pd.read_csv(BRN_path, encoding='utf_8_sig')

brn = df['BR Number']

brn_list = ""

for b in brn:
    brn_list += f'`Certificate_No.` LIKE "{b}%" OR '


check_list = f' WHERE {brn_list}'
query = check_list[:-3]





try:
    # Connect to the database
    conn = sqlite3.connect(db_path)
except sqlite3.Error as e:
    print('Database not found:', str(e))
    exit()

# Get a list of table names in the database
table_list = []
cursor = conn.cursor()
for table_name in cursor.execute("SELECT name FROM sqlite_master WHERE type = 'table'"):
    table_list.append(table_name)

# Initialize a list to store the results
results_list = []

# Iterate over each table
for table in table_list:
    try:
        # Execute the query to retrieve rows with Certificate_No. matching the pattern
        cursor.execute(f'SELECT `Event_ID`, `Place_of_Business_Registration`, `Status`,`Certificate_No.`, `Date_of_Expiry`, `Company_name`, `Name_of_Business_/_Corporation_BR_Chi`, `Address_BR_-_Unit_/_Flat_/_Shop_and_Floor`, `Address_BR_-_Building`, `Address_BR_-_Street`, `Address_BR_-_District` FROM `{table[0]}` {query} ORDER BY `Certificate_No.` ASC')
        results = cursor.fetchall()
        for row in results:
            # Create a dictionary entry for each row
            row_dict = {}
            for i, header in enumerate(cursor.description):
                row_dict[header[0]] = row[i]
            results_list.append(row_dict)
    except sqlite3.Error:
        continue

# Close the database connection
conn.close()

# Create a DataFrame from the results list
df = pd.DataFrame(results_list)
df2 = df.drop_duplicates(keep='first')
df3 = df2.sort_values('Certificate_No.')

filep = r'C:\Users\hofong\Desktop\python\1a Python working station\Database\rejoinder.xlsx'
# Print the resulting DataFrame
df3.to_excel(filep, index=False)




finaldf = pd.read_excel(filep)

# Count occurrences of 'Certificate_No.'
certificate_counts = finaldf['Certificate_No.'].value_counts()

# Create a new column 'Times' and assign the counts
finaldf['Times'] = finaldf['Certificate_No.'].map(certificate_counts)

# Define the desired column sequence
column_sequence = [
    'Times',
    'Event_ID',
    'Place_of_Business_Registration',
    'Status',
    'Certificate_No.',
    'Date_of_Expiry',
    'Company_name',
    'Name_of_Business_/_Corporation_BR_Chi',
    'Address_BR_-_Unit_/_Flat_/_Shop_and_Floor',
    'Address_BR_-_Building',
    'Address_BR_-_Street',
    'Address_BR_-_District'
]

# Reorder columns based on the specified sequence
finaldf = finaldf[column_sequence]

# Save the updated DataFrame to a new Excel file
finaldf.to_excel(filep, index=False)

# Print a success message
print("Columns rearranged and saved to test.xlsx")

finaldf

Columns rearranged and saved to test.xlsx


,Rejoining,Event_ID,Place_of_Business_Registration,Status,Certificate_No.,Date_of_Expiry,Company_name,Name_of_Business_/_Corporation_BR_Chi,Address_BR_-_Unit_/_Flat_/_Shop_and_Floor,Address_BR_-_Building,Address_BR_-_Street,Address_BR_-_District
0,6,TDC-071,HK,Body Corporate,33430452-000,2024-03-12,A and P Productions Limited,嘉宏製品有限公司,UNIT 13 12/F,ELITE IND CTR,883 CHEUNG SHA WAN ROAD,NaN
1,6,TDC-007,HK,Body Corporate,33430452-000,2022-03-11,A and P Productions Limited,嘉宏製品有限公司,UNIT 13 12/F,ELITE IND CENTRE,883 CHEUNG SHA WAN ROAD,KL
2,6,TDC-031,HK,Body Corporate,33430452-000,2023-03-12,A AND P PRODUCTIONS LTD,嘉宏製品有限公司,UNIT 13 12/F,ELITE IND CTR,883 CHEUNG SHA WAN ROAD,NaN
3,6,TDC-079,HK,Body Corporate,33430452-000,2024-03-12,A AND P PRODUCTIONS LTD,嘉宏製品有限公司,UNIT 13 12/F,ELITE IND CTR,883 CHEUNG SHA WAN ROAD,KL
4,6,TDC-029,HK,Body Corporate,33430452-000,2022-03-12,A and P Productions Limited,嘉宏製品有限公司,UNIT 13 12/F,ELITE IND CENTRE,883 CHEUNG SHA WAN ROAD,KL
5,6,TDC-066,HK,Body Corporate,33430452-000,2023-03-12,A and P Productions Limited,嘉宏製品有限公司,UNIT 13 12/F,ELITE IND CTR,883 CHEUNG SHA WAN ROAD,0
6,1,TDC-096,Mainland China,NaN,91440300MA5ED3U84W-,NaN,深圳市奧新科技有限公司,NaN,NaN,NaN,NaN,NaN
